## 使用 POST 方法來爬取全國 7-11 各門市的資訊
### 本範例欲爬的目標網站：[7- 11 ibon 便利生活站](http://www.ibon.com.tw/default.aspx) （[7-11 ibon 便利生活站→門市查詢](http://www.ibon.com.tw/retail_inquiry.aspx#gsc.tab=0)）
### 補充參考：<br><br>1. HTTP 協定中的 GET 與 POST [[1]](https://blog.toright.com/posts/1203/%E6%B7%BA%E8%AB%87-http-method%EF%BC%9A%E8%A1%A8%E5%96%AE%E4%B8%AD%E7%9A%84-get-%E8%88%87-post-%E6%9C%89%E4%BB%80%E9%BA%BC%E5%B7%AE%E5%88%A5%EF%BC%9F.html) [[2]](https://www.w3schools.com/tags/ref_httpmethods.asp) [[3]](https://www.guru99.com/difference-get-post-http.html) [[4*]](https://www.codesnail.com/introduction-to-web-detailed-explanation/) [[5*]](https://codertw.com/%E5%89%8D%E7%AB%AF%E9%96%8B%E7%99%BC/24848/)<br><br>2. Web 表單（.aspx & ajax）基礎概念參考 [[1]](https://ithelp.ithome.com.tw/articles/10213192) [[2]](https://ithelp.ithome.com.tw/articles/10213242) [[3]](https://www.uj5u.com/qianduan/261211.html) [[4]](https://www.itread01.com/content/1549292247.html) [[5]](https://www.learncodewithmike.com/2020/10/scraping-ajax-websites-using-python.html) [[6]](https://iter01.com/48414.html) [[7]](https://tw.alphacamp.co/blog/ajax-asynchronous-request) [[8]](https://kknews.cc/zh-tw/code/ppg62n2.html) [[9]](https://blog.csdn.net/guofang9410/article/details/69696814)

In [ ]:
#
# 可以去掉 Python 輸出時，因為軟體版本所引起的警告的警告。
#
import warnings

warnings.filterwarnings('ignore')

In [ ]:
import requests

from bs4 import BeautifulSoup

import pandas as pd

#
# 建立一個縣市的 list，以方便抓取 7-11 各門市資訊
#
city = ['基隆市', '台北市', '新北市', '桃園市', '新竹市', '新竹縣', '苗栗縣', 
        '台中市', '彰化縣', '雲林縣', '南投縣', '嘉義縣', '嘉義市', '台南市',
        '高雄市', '屏東縣', '台東縣', '花蓮縣', '宜蘭縣', '連江縣', '金門縣', 
        '澎湖縣', '南海諸島']


#
# 使用者自訂 http browser request header 中的 user-agent 內容
#
headers = { 
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) \
             AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.54 Safari/537.36'
          } 


#
# 使用迴圈來依序取得每一個城市的門市資訊
#
for index, city in enumerate(city):
    
    #
    # 剛剛在開發者模式觀察到的 Post 發出的資訊是那些
    #
    payload = {
        'strTargetField':'COUNTY',
        'strKeyWords':'%s' % city
    }
    
    res = requests.post('https://www.ibon.com.tw/retail_inquiry_ajax.aspx', 
                        data = payload,
                        headers = headers)
    
    #
    # 第一次迴圈建立 dataframe，並將城市填入。資料的形式是 table，所以直接使用 read_html 快速拿下!
    #
    if index == 0:
        
        #
        # 因 pd.read_html 可能會傳回多個 DataFrame（HTML 網頁上的所有表格物件）
        # 而本例中只有一個 DataFrame，其索引值為 0
        #
        # pd.read_html 中 header = 0 是想用網頁中表格的第一行當成欄位名稱（column name） 
        #
        df_7_11_store = pd.read_html(res.text, header = 0)[0] 
                                     
        df_7_11_store['縣市'] = city   # 加入一個名為 '縣市' 的新欄位
    
        #print(len(df_7_11_store))        
        
        #
        # 列印出目前處理的進度
        #
        # The star indicates that the field width (here: 5) will be 
        # passed as a parameter to print
        #
        print('\n\n%d)%-*s現有 %d 個分店\n' % \
              (index + 1, 5, city, pd.read_html(res.text, header = 0)[0].shape[0]))
        
        #
        # 存取與列印第一個縣市的分店訊息
        #      
        df_7_11_store.index += 1
        
        print(df_7_11_store)
        
   
    
    #
    # 第二次迴圈以上就將資訊直接 append 到 dataframe 裡
    #
    if index > 0:
   
        #
        # 存取與列印每一個縣市的分店訊息
        #
        df_7_11_store_other_info = pd.read_html(res.text, header = 0)[0]
        
        df_7_11_store_other_info.index += 1

        #
        # 列印出目前處理的進度
        #
        # The star indicates that the field width (here: 5) will be passed as a parameter to print
        #
        print('\n\n%d)%-*s現有 %d 個分店\n' % (index + 1, 5, city, pd.read_html(res.text, header = 0)[0].shape[0]))
        
        print(df_7_11_store_other_info)
        
        #
        # 合併每一個縣市的分店訊息表格
        #
        df_7_11_store_other_info['縣市'] = city   # 加入一個名為 '縣市' 的新欄位
                      
        df_7_11_store = pd.concat([df_7_11_store, df_7_11_store_other_info])  # 新的寫法
        #df_7_11_store = df_7_11_store.append(df_7_11_store_other_info)  # 舊的寫法，但仍可以使用

#### pd.read_html（）內部參數的一些值得看的參考 [[1]](https://www.facebook.com/groups/pythontw/posts/10159737916898438/) [[2]](https://ithelp.ithome.com.tw/questions/10193155?sc=pt) [[3]](https://juejin.cn/s/pandas%20read_html%20header%20example)

In [ ]:
#
# 將資料輸出成 Excel 檔案
#
df_7_11_store.to_excel('7-11-sales-output.xlsx', 
                       encoding = "utf8", 
                       index = False)

'''
!pip install xlsxwriter

import xlsxwriter

xl = pd.ExcelFile('7-11-sales-output.xlsx')

print(xl.sheet_names)
'''

#
# 將資料從 Excel 檔案中讀出
#
df = pd.read_excel('7-11-sales-output.xlsx')


#
# 重設 dataframe 的索引值從 1 開始
#
df.index += 1

print()

print(df.head(), '\n')    # 只列印出前五筆資料(預設值)


#print(df)          # 列印出全部的資料

#print(df.head(10), '\n') # 只列印出前十筆資料

print(df.tail(), '\n')   # 只列印出最後五筆資料

**[可線上讀取 Excel 檔案內容的工具](https://products.aspose.app/cells/viewer)**

In [ ]:
#
# 另存成一個 html 檔，並能成功的顯示中文
#
html = df_7_11_store.to_html() 

with open("7-11-sales-output.html", "w", encoding = "utf8") as file: 
    
    #
    # 將 <meta charset = "UTF-8">\n 此行強制寫入瀏覽器的開頭內，
    # 其目的是指定瀏覽器使用 UTF8 來解讀    
    #
    file.writelines('<meta charset = "UTF-8">\n')  
    
    file.write(html)